# Fasting Pandas - A guide into optimizing your analytical processing 

### Part 3

---

We now dive into the realm of files. We want to show we are better than those pesky excel and csv pencil pushers. This is an important step to start giving an impression of actually knowing something.

Before going any further, I will outright tell you I'm not going to talk about databases. This is out of the scope for this tutorial, for now. First buy me some coffee and dinner and I will think about it.

All right, so why did we look into parsing before file management? The main reason is that parsing will have a direct impact on the efficiency and benefits of file formats, so let's start by generating a dataframe. We will now increment our dataset to 20 million rows.

Then, a quick comparison on memory optimization.

In [1]:
from importlib import reload
import fasting_pandas as fp
from fasting_pandas import datasets as ds
reload(ds)
reload(fp)
import pandas as pd
import numpy as np
import os

In [2]:
df = fp.generate_results(20_000_000)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000000 entries, 0 to 19999999
Data columns (total 6 columns):
 #   Column  Dtype         
---  ------  -----         
 0   size    object        
 1   age     int32         
 2   team    object        
 3   result  object        
 4   date    datetime64[ns]
 5   prob    float64       
dtypes: datetime64[ns](1), float64(1), int32(1), object(3)
memory usage: 839.2+ MB


In [3]:
fp.set_dtypes_for_results(df)
df.drop(axis = 1, columns = 'result', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000000 entries, 0 to 19999999
Data columns (total 6 columns):
 #   Column  Dtype         
---  ------  -----         
 0   size    category      
 1   age     int8          
 2   team    category      
 3   date    datetime64[ns]
 4   prob    float16       
 5   win     bool          
dtypes: bool(1), category(2), datetime64[ns](1), float16(1), int8(1)
memory usage: 267.0 MB


Ok so parsing reduced our dataset memory from 840mb to 250mb. All good, but now we want to save our work for future transformations or just for sharing. What now?

## CSV

CSVs are in the forefront of data analytics. Chances are you have encountered one of these bad boys in the wild, and you had to read it. That is if you are lucky. There are stories of having to read multiple csvs from a folder and having to merge them into a single dataframe. Sometimes I think we are paying for mistakes we made on a past life.

Anywho, for comparison purposes we will measure how much time it takes to save and read csvs. Also we will look into the size of the files to understand how much physical memory we are wasting.

I want to keep things simple so quick disclaimer. Check file command is different depending on the OS so:
- Windows
 %ls -GFlash [filename]

- Linux:
 !ls -GFlash [filename]

In [7]:
#%%timeit
# Disclosure. This takes a long time to execute since it loops 7 times. Don't waste your life on this and just take my word. Or don't, you do you.
df.to_csv(os.path.join(fp.DATA_DIR,'dataset.csv'), index = False)

In [8]:
%ls -GFlash data\dataset.csv

 Volume in drive D is Work_1TB
 Volume Serial Number is 9C92-468C

 Directory of d:\Proyectos\datakai\projects\github\datakaicr\public\fasting-pandas


 Directory of d:\Proyectos\datakai\projects\github\datakaicr\public\fasting-pandas\data

03/25/2023  11:50 PM       786,079,237 dataset.csv
               1 File(s)    786,079,237 bytes
               0 Dir(s)  606,673,330,176 bytes free


File Not Found


In [9]:
%timeit df_csv = pd.read_csv(os.path.join(fp.DATA_DIR,'dataset.csv'))

8.94 s ± 148 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
df_csv = pd.read_csv(os.path.join(fp.DATA_DIR,'dataset.csv'))

In [11]:
df_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000000 entries, 0 to 19999999
Data columns (total 6 columns):
 #   Column  Dtype  
---  ------  -----  
 0   size    object 
 1   age     int64  
 2   team    object 
 3   date    object 
 4   prob    float64
 5   win     bool   
dtypes: bool(1), float64(1), int64(1), object(3)
memory usage: 782.0+ MB


Well well well, so lots of interesting things going on with the CSVs.

First interesting fact is they are trash.

It took 50 seconds to save, the size of the file is 750mb, around 10 seconds to read and it caused havoc on our carefully crafted datatypes and now we are stuck with a grotesque csv that is consuming 780mb of RAM.

Lesson, avoid CSVs like the plague unless you don't have a choice. Thankfully, we don't need to use them anymore. Ok thank you byeeeee

In [12]:
try:
    del df_csv
except NameError as e:
    print(e)


In [13]:
try:
    os.remove('data/dataset.csv')
except FileNotFoundError as e:
    print(e)
except NameError as e:
    print(e)


## Pickle



In [14]:
%%timeit
df.to_pickle(os.path.join(fp.DATA_DIR,'dataset.pickle'))

1.08 s ± 33.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%ls -GFlash data\dataset.pickle

 Volume in drive D is Work_1TB
 Volume Serial Number is 9C92-468C

 Directory of d:\Proyectos\datakai\projects\github\datakaicr\public\fasting-pandas


 Directory of d:\Proyectos\datakai\projects\github\datakaicr\public\fasting-pandas\data

03/25/2023  11:52 PM       280,001,673 dataset.pickle
               1 File(s)    280,001,673 bytes
               0 Dir(s)  607,459,409,920 bytes free


File Not Found


In [16]:
%timeit df_pickle = pd.read_pickle(os.path.join(fp.DATA_DIR,'dataset.pickle'))

100 ms ± 4.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
df_pickle = pd.read_pickle(os.path.join(fp.DATA_DIR,'dataset.pickle'))

In [18]:
df_pickle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000000 entries, 0 to 19999999
Data columns (total 6 columns):
 #   Column  Dtype         
---  ------  -----         
 0   size    category      
 1   age     int8          
 2   team    category      
 3   date    datetime64[ns]
 4   prob    float16       
 5   win     bool          
dtypes: bool(1), category(2), datetime64[ns](1), float16(1), int8(1)
memory usage: 267.0 MB


Pickle files seem so much more promising. Lets point out the benefits:

1. File size is significantly lower. We went down from 780mb to 280mb. Compression is good for going downhill, and for saving your files.
2. They are much faster to write. 50 to 1 second drop.
3. They are take 100 ms to read.
4. Last but not least, they respected the datatype parsing!!



## Parquet

In [19]:
%%timeit
df.to_parquet(os.path.join(fp.DATA_DIR,'dataset.parquet'))

ArrowNotImplementedError: Unhandled type for Arrow to Parquet schema conversion: halffloat

I felt this error on purpose. Parquet files don't work nicely with any float below 32 bits, so we need to adjust.

In [20]:
df['prob'] = df['prob'].astype('float32')

In [21]:
%%timeit
df.to_parquet(os.path.join(fp.DATA_DIR,'dataset.parquet'))

2.28 s ± 43.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%timeit df_parquet = pd.read_parquet(os.path.join(fp.DATA_DIR,'dataset.parquet'))

402 ms ± 9.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
df.to_parquet('dataset.parquet')
df_parquet = pd.read_parquet('dataset.parquet')
df.info()

In [ ]:
df = fp.generate_results(20_000_000)
fp.set_dtypes_for_results(df)
df.drop(axis = 1, columns = 'result', inplace=True)
df.to_parquet('dataset_parquet.parquet')
df = pd.read_parquet('dataset_parquet.parquet')
df.info()

In [ ]:
%ls -GFlash dataset_parquet.parquet

In [ ]:
df = fp.generate_results(20_000_000)
# fp.set_dtypes_for_results(df)
# df.drop(axis = 1, columns = 'result', inplace=True)
df.to_parquet('dataset_parquet.parquet')
df = pd.read_parquet('dataset_parquet.parquet')
df.info()